# Chapter 4: Text Classification - Medium Tasks

This notebook focuses on building practical text classifiers. You'll create custom multi-class sentiment classifiers, evaluate performance with limited training data, implement confidence-based classification with uncertainty handling, and perform systematic failure analysis. These skills are crucial for real-world NLP applications where data and perfect accuracy are often limited.


---

## Setup

Run all cells in this section to set up the environment and load necessary data.


### [OPTIONAL] - Installing Packages on <img src="https://colab.google/static/images/icons/colab.png" width=100>


If you are viewing this notebook on Google Colab (or any other cloud vendor), you need to **uncomment and run** the following codeblock to install the dependencies for this chapter:

---

 **NOTE**: We will want to use a GPU to run the examples in this notebook. In Google Colab, go to
**Runtime > Change runtime type > Hardware accelerator > GPU > GPU type > T4**.

---


In [1]:
 %%capture
!pip install transformers sentence-transformers openai
!pip install -U datasets

### Data Loading


In [2]:
from datasets import load_dataset

# Load our data
data = load_dataset("rotten_tomatoes")
data

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

### Helper Functions


In [6]:
from sklearn.metrics import classification_report

def evaluate_performance(y_true, y_pred):
    """Create and print the classification report"""
    performance = classification_report(
        y_true, y_pred,
        target_names=["Negative Review", "Positive Review"]
    )
    print(performance)

## Your Turn - Text Classification Experiments

Run each task first to see the baseline results. Follow the instructions to modify and experiment.

This section is divided into EASY, MEDIUM, & HARD.

---

## Medium Tasks


### Medium Tasks - Building Real Classifiers

These tasks require more modification and experimentation. You'll build complete classification systems.

Run the code to see how 5-level classification works. Then try adding a 6th category.


In [ ]:
#### Medium Task 1: Multi-Class Sentiment Classification

In this task, you'll build a sentiment classifier with 5 different categories (from extremely negative to extremely positive) instead of just binary positive/negative.

**What to do:**
1. Run the cells below to see baseline 5-level classification
2. Observe which reviews are uncertain (low margin between top predictions)
3. Try uncommenting the 6-level version to add more granularity
4. Compare how predictions change with more categories

Set up the 5 sentiment categories and compute embeddings:

In [ ]:
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report
from datasets import load_dataset
import numpy as np

data = load_dataset("rotten_tomatoes")

train_size = 1000  # TODO: Try different values: 100, 500, 1000, 2000, 5000
test_size = 300

train_subset = data["train"].shuffle(seed=42).select(range(min(train_size, len(data["train"]))))
test_subset = data["test"].shuffle(seed=42).select(range(test_size))

print("="*80)
print(f"EXPERIMENT: Training Size = {train_size}")
print("="*80)

Classify each review and show confidence scores:

In [ ]:
#### Medium Task 2: Classifier Performance with Limited Training Data

How much training data do you really need? This task compares two approaches: a task-specific model pre-trained on sentiment versus an embedding classifier trained from scratch on your data. Start by running with 1000 samples, then experiment with different training sizes like 100, 500, or 2000. Watch for the crossover point where the embedding classifier catches up to the pre-trained model and record your results in the table at the end.

As you can see, the classifier assigns each review to one of the 5 sentiment categories. The **margin** (difference between top 2 predictions) indicates confidence - large margins (>0.15) mean the model is confident, while small margins (<0.05) indicate uncertainty. Reviews with extreme language ("best ever", "terrible") have higher confidence, while moderate reviews ("pretty good", "quite bad") show more uncertainty.

Analyze which categories are most similar to each other:

In [ ]:
print("\n" + "="*80)
print("CATEGORY CONFUSION ANALYSIS")
print("="*80)

label_similarity = cosine_similarity(label_embeddings)

print(f"{'Category Pair':<60s} {'Similarity':<12s}")
print("-"*75)

confusions = []
for i in range(len(sentiment_labels)):
    for j in range(i+1, len(sentiment_labels)):
        sim = label_similarity[i][j]
        confusions.append((i, j, sim))

for i, j, sim in sorted(confusions, key=lambda x: x[2], reverse=True)[:10]:
    pair_name = f"{sentiment_labels[i]} <-> {sentiment_labels[j]}"
    marker = " " if sim > 0.7 else ""
    print(f"{marker}{pair_name:<60s} {sim:.3f}")

Notice that adjacent categories (like "somewhat negative" and "neutral") tend to have higher similarity scores, which explains why the model sometimes confuses them. Categories with similarity > 0.7 are particularly prone to confusion.

In [ ]:
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report
from datasets import load_dataset
import numpy as np

data = load_dataset("rotten_tomatoes")

train_size = 1000  # TODO: Try different values: 100, 500, 1000, 2000, 5000
test_size = 300

train_subset = data["train"].shuffle(seed=42).select(range(min(train_size, len(data["train"]))))
test_subset = data["test"].shuffle(seed=42).select(range(test_size))

print("="*80)
print(f"EXPERIMENT: Training Size = {train_size}")
print("="*80)

#### Medium Task 2: Classifier Performance with Limited Training Data

Try different training sizes (100, 500, 1000, 2000) and fill in the results table.


In [ ]:
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report
from datasets import load_dataset
import numpy as np

data = load_dataset("rotten_tomatoes")

# TODO: Try different values: 100, 500, 1000, 2000, 5000
train_size = 1000
test_size = 300

train_subset = data["train"].shuffle(seed=42).select(range(min(train_size, len(data["train"]))))
test_subset = data["test"].shuffle(seed=42).select(range(test_size))

print("="*80)
print(f"EXPERIMENT: Training Size = {train_size}")
print("="*80)

#### Medium Task 3: Confidence-Based Classifier with Uncertainty Handling

In production, refusing a prediction beats making a wrong one. Here's the key insight: when your model is uncertain, it should say "I don't know" rather than guessing. This creates a trade-off between coverage (how many predictions you make) and accuracy (how often you're right).

Try this:
- Run with threshold of 0.15 first
- Test 0.05, 0.30, and 0.50 to see how the trade-off shifts
- Check the uncertain cases (typically have hedging language)
- Experiment with the alternative uncertainty measure

In [ ]:
print("\n[1/2] Testing Task-Specific Model...")

task_model = pipeline(
    model="cardiffnlp/twitter-roberta-base-sentiment-latest",
    tokenizer="cardiffnlp/twitter-roberta-base-sentiment-latest",
    return_all_scores=True,
    device=-1
)

y_pred_task = []
for text in test_subset["text"]:
    output = task_model(text)[0]
    neg_score = output[0]["score"]
    pos_score = output[2]["score"]
    y_pred_task.append(1 if pos_score > neg_score else 0)

task_f1 = f1_score(test_subset["label"], y_pred_task, average='weighted')
print(f" Task-Specific Model F1: {task_f1:.4f}")

Train the embedding-based classifier on your labeled data:

In [ ]:
print(f"\n[2/2] Training Embedding Classifier on {train_size} samples...")

embedding_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

train_embeddings = embedding_model.encode(train_subset["text"], show_progress_bar=False)
test_embeddings = embedding_model.encode(test_subset["text"], show_progress_bar=False)

clf = LogisticRegression(random_state=42, max_iter=1000)
clf.fit(train_embeddings, train_subset["label"])

y_pred_embed = clf.predict(test_embeddings)
embed_f1 = f1_score(test_subset["label"], y_pred_embed, average='weighted')

print(f" Embedding Classifier F1: {embed_f1:.4f}")

Compare the two approaches and show example predictions:

In [ ]:
print("\n" + "="*80)
print("RESULTS SUMMARY")
print("="*80)
print(f"Training samples used: {train_size}")
print(f"\nTask-Specific (pre-trained):  F1 = {task_f1:.4f}")
print(f"Embedding + Classifier:       F1 = {embed_f1:.4f}")
print(f"Difference:                       {embed_f1 - task_f1:+.4f}")

if embed_f1 > task_f1:
    print(f"\n Embedding approach WINS with {train_size} samples!")
elif embed_f1 > task_f1 - 0.01:
    print(f"\n Essentially TIED")
else:
    print(f"\n Task-specific model wins")

# Show example predictions
print("\n" + "="*80)
print("EXAMPLE PREDICTIONS (first 5)")
print("="*80)

for i in range(5):
    true_label = "Positive" if test_subset["label"][i] == 1 else "Negative"
    task_pred = "Positive" if y_pred_task[i] == 1 else "Negative"
    embed_pred = "Positive" if y_pred_embed[i] == 1 else "Negative"

    task_correct = "" if y_pred_task[i] == test_subset["label"][i] else ""
    embed_correct = "" if y_pred_embed[i] == test_subset["label"][i] else ""

    print(f"\n{i+1}. '{test_subset['text'][i][:60]}...'")
    print(f"   True: {true_label}")
    print(f"   Task-Specific: {task_pred} {task_correct}")
    print(f"   Embedding:     {embed_pred} {embed_correct}")

print("\n" + "="*80)
print("TODO: Record your results")
print("="*80)
print(f"Current: | {train_size:<10} | {task_f1:.4f}  | {embed_f1:.4f}       | {'Embed' if embed_f1 > task_f1 else 'Task':<11} |")

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Reviews with varying clarity levels
test_reviews = [
    "Absolutely fantastic! Best movie ever!",
    "Pretty good, I liked it",
    "It was fine, nothing special",
    "Not bad but not great either",
    "Quite disappointing",
    "Terrible! Complete waste of time!",
    "The movie had some interesting moments",
    "Outstanding performances all around!",
]

y_true = [1, 1, 0, 0, 0, 0, 1, 1]  # True labels: 1=positive, 0=negative

labels = ["A negative movie review", "A positive movie review"]

confidence_threshold = 0.15  # TODO: Try values: 0.05, 0.15, 0.30, 0.50

label_embeddings = model.encode(labels)
review_embeddings = model.encode(test_reviews)
sim_matrix = cosine_similarity(review_embeddings, label_embeddings)

def calculate_margin(similarities):
    """Margin = difference between top two predictions"""
    sorted_sims = np.sort(similarities)[::-1]
    margin = sorted_sims[0] - sorted_sims[1]
    return margin

# TODO: Try this alternative uncertainty measure
# def calculate_margin(similarities):
#     """Alternative: Use absolute confidence in top prediction"""
#     max_confidence = np.max(similarities)
#     margin = max_confidence - 0.5
#     return margin

In [ ]:
# Calculate metrics
made_predictions = [r for r in results if r['pred'] is not None]
uncertain_cases = [r for r in results if r['pred'] is None]
correct_predictions = [r for r in made_predictions if r['pred'] == r['true']]

total = len(results)
n_predicted = len(made_predictions)
n_uncertain = len(uncertain_cases)
n_correct = len(correct_predictions)

coverage = n_predicted / total
accuracy = n_correct / n_predicted if n_predicted > 0 else 0

print("\n" + "="*80)
print("PERFORMANCE ANALYSIS")
print("="*80)

print(f"\nCoverage: {n_predicted}/{total} = {coverage:.1%}")
print(f"   Made predictions for {n_predicted} reviews")
print(f"   Refused to predict on {n_uncertain} reviews")

print(f"\nAccuracy (on predictions made): {n_correct}/{n_predicted} = {accuracy:.1%}")
print(f"   Of the {n_predicted} predictions, {n_correct} were correct")

print(f"\nTrade-off Analysis:")
print(f"  Threshold = {confidence_threshold}")
print(f"   Higher threshold = fewer predictions but higher accuracy")
print(f"   Lower threshold = more predictions but lower accuracy")

if n_uncertain > 0:
    print(f"\n" + "-"*80)
    print(f"UNCERTAIN CASES (margin < {confidence_threshold}):")
    print("-"*80)
    for r in uncertain_cases:
        print(f"  • '{r['review']}'")
        print(f"    Margin: {r['margin']:.3f} (too close to call)")

In [ ]:
#### Medium Task 4: Systematic Classifier Failure Analysis

Understanding where and why your classifier fails is crucial for improvement. Run the analysis to see overall error patterns, examine high-confidence errors (the most surprising mistakes), and test edge cases like sarcasm and mixed sentiment. Add your own challenging test cases, then propose improvements based on what you find.

In [ ]:
#### Medium Task 4: Systematic Classifier Failure Analysis

Looking at the overall error patterns reveals where your classifier struggles. Run the analysis to examine high-confidence errors (the most surprising mistakes) and test edge cases like sarcasm and mixed sentiment. Add your own challenging test cases, then propose improvements based on what you find.

### Questions

1. At what training size did embedding classifier match the task-specific model?

2. Were there cases where one model was correct and the other wrong?

3. Is 100 samples enough labeled data?


#### Medium Task 3: Confidence-Based Classifier

Run with threshold=0.15, then try 0.05 and 0.30 to see the trade-offs.


In [32]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Reviews with varying levels of clarity
test_reviews = [
    "Absolutely fantastic! Best movie ever!",           # Clear positive
    "Pretty good, I liked it",                          # Weak positive
    "It was fine, nothing special",                     # Ambiguous
    "Not bad but not great either",                     # Very ambiguous
    "Quite disappointing",                              # Weak negative
    "Terrible! Complete waste of time!",                # Clear negative
    "The movie had some interesting moments",           # Ambiguous positive
    "Outstanding performances all around!",             # Clear positive
]

# True labels (for evaluation)
y_true = [1, 1, 0, 0, 0, 0, 1, 1]  # 1=positive, 0=negative

labels = ["A negative movie review", "A positive movie review"]

# TODO: EXPERIMENT WITH THIS - Try: 0.05, 0.15, 0.30, 0.50
confidence_threshold = 0.15

label_embeddings = model.encode(labels)
review_embeddings = model.encode(test_reviews)
sim_matrix = cosine_similarity(review_embeddings, label_embeddings)

def calculate_margin(similarities):
    """
    Margin = difference between top two predictions
    Small margin = uncertain (predictions are close)
    """
    sorted_sims = np.sort(similarities)[::-1]
    margin = sorted_sims[0] - sorted_sims[1]
    return margin

# TODO: After first run, uncomment this alternative uncertainty measure:
# def calculate_margin(similarities):
#     """
#     Alternative: Use absolute confidence in top prediction
#     Low confidence = uncertain
#     """
#     max_confidence = np.max(similarities)
#     # Convert to margin-like score (higher = more certain)
#     # If max is 0.6, margin = 0.6 - 0.5 = 0.1 (uncertain)
#     # If max is 0.9, margin = 0.9 - 0.5 = 0.4 (certain)
#     margin = max_confidence - 0.5
#     return margin

# Classify with confidence threshold
results = []
predictions = []
confidences = []

print("="*80)
print(f"CONFIDENCE-BASED CLASSIFICATION (threshold={confidence_threshold})")
print("="*80)

for i, review in enumerate(test_reviews):
    similarities = sim_matrix[i]
    predicted_idx = np.argmax(similarities)
    top_confidence = similarities[predicted_idx]
    margin = calculate_margin(similarities)

    # Decision: predict only if confident enough
    if margin >= confidence_threshold:
        prediction = predicted_idx
        status = "PREDICTED"
        predictions.append(prediction)
    else:
        prediction = None
        status = "UNCERTAIN"
        predictions.append(None)

    true_label = "Positive" if y_true[i] == 1 else "Negative"
    pred_label = labels[predicted_idx] if prediction is not None else "UNCERTAIN"

    print(f"\n{i+1}. '{review}'")
    print(f"   True label: {true_label}")
    print(f"   Prediction: {pred_label}")
    print(f"   Top confidence: {top_confidence:.3f}")
    print(f"   Margin: {margin:.3f} {' Above threshold' if margin >= confidence_threshold else ' Below threshold'}")
    print(f"   Status: {status}", end="")

    if prediction is not None:
        correct = prediction == y_true[i]
        print(f" - {' CORRECT' if correct else ' INCORRECT'}")
    else:
        print()

    results.append({
        'review': review,
        'true': y_true[i],
        'pred': prediction,
        'margin': margin,
        'status': status
    })

# Calculate metrics
print("\n" + "="*80)
print("PERFORMANCE ANALYSIS")
print("="*80)

made_predictions = [r for r in results if r['pred'] is not None]
uncertain_cases = [r for r in results if r['pred'] is None]
correct_predictions = [r for r in made_predictions if r['pred'] == r['true']]

total = len(results)
n_predicted = len(made_predictions)
n_uncertain = len(uncertain_cases)
n_correct = len(correct_predictions)

coverage = n_predicted / total
accuracy = n_correct / n_predicted if n_predicted > 0 else 0

print(f"\nCoverage: {n_predicted}/{total} = {coverage:.1%}")
print(f"   Made predictions for {n_predicted} reviews")
print(f"   Refused to predict on {n_uncertain} reviews")

print(f"\nAccuracy (on predictions made): {n_correct}/{n_predicted} = {accuracy:.1%}")
print(f"   Of the {n_predicted} predictions, {n_correct} were correct")

print(f"\nTrade-off Analysis:")
print(f"  Threshold = {confidence_threshold}")
print(f"   Higher threshold = fewer predictions but higher accuracy")
print(f"   Lower threshold = more predictions but lower accuracy")

# Show which reviews were uncertain
if n_uncertain > 0:
    print(f"\n" + "-"*80)
    print(f"UNCERTAIN CASES (margin < {confidence_threshold}):")
    print("-"*80)
    for r in uncertain_cases:
        print(f"  • '{r['review']}'")
        print(f"    Margin: {r['margin']:.3f} (too close to call)")

# TODO: After experimenting with thresholds, analyze the trade-off
print("\n" + "="*80)
print("EXPERIMENT LOG - Fill this in as you try different thresholds:")
print("="*80)
print("| Threshold | Coverage | Accuracy | Notes                    |")
print("|-----------|----------|----------|--------------------------|")
print("| 0.05      | ??.?%    | ??.?%    | ?                        |")
print("| 0.15      | ??.?%    | ??.?%    | ?                        |")
print("| 0.30      | ??.?%    | ??.?%    | ?                        |")
print("| 0.50      | ??.?%    | ??.?%    | ?                        |")
print()
print(f"Current:    | {confidence_threshold:<9.2f} | {coverage*100:>5.1f}%    | {accuracy*100:>5.1f}%    |")

CONFIDENCE-BASED CLASSIFICATION (threshold=0.15)

1. 'Absolutely fantastic! Best movie ever!'
   True label: Positive
   Prediction: UNCERTAIN
   Top confidence: 0.451
   Margin: 0.092 ✗ Below threshold
   Status: UNCERTAIN

2. 'Pretty good, I liked it'
   True label: Positive
   Prediction: UNCERTAIN
   Top confidence: 0.410
   Margin: 0.018 ✗ Below threshold
   Status: UNCERTAIN

3. 'It was fine, nothing special'
   True label: Negative
   Prediction: UNCERTAIN
   Top confidence: 0.418
   Margin: 0.051 ✗ Below threshold
   Status: UNCERTAIN

4. 'Not bad but not great either'
   True label: Negative
   Prediction: UNCERTAIN
   Top confidence: 0.414
   Margin: 0.053 ✗ Below threshold
   Status: UNCERTAIN

5. 'Quite disappointing'
   True label: Negative
   Prediction: UNCERTAIN
   Top confidence: 0.354
   Margin: 0.080 ✗ Below threshold
   Status: UNCERTAIN

6. 'Terrible! Complete waste of time!'
   True label: Negative
   Prediction: UNCERTAIN
   Top confidence: 0.397
   Margin: 0.121

### Questions

1. What do uncertain reviews have in common? Are they using hedging language like "kind of" or "somewhat"?

2. Compare results at threshold=0.05 vs 0.30. Describe the coverage vs accuracy trade-off. When would you want high coverage vs high accuracy?

3. How could you use confidence-based prediction in production? What should a system do when the model is uncertain?

#### Medium Task 4: Classifier Failure Analysis

Train the classifier and see what kinds of reviews it gets wrong. Then add your own test cases.


In [33]:
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from datasets import load_dataset
import numpy as np

# Load data
data = load_dataset("rotten_tomatoes")
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Use subset for faster experimentation
train_subset = data["train"].shuffle(seed=42).select(range(1000))
test_subset = data["test"].shuffle(seed=42).select(range(200))

# Train classifier
print("Training classifier on 1000 movie reviews...")
train_embeddings = model.encode(train_subset["text"], show_progress_bar=False)
test_embeddings = model.encode(test_subset["text"], show_progress_bar=False)

clf = LogisticRegression(random_state=42, max_iter=1000)
clf.fit(train_embeddings, train_subset["label"])

# Get predictions
predictions = clf.predict(test_embeddings)
probabilities = clf.predict_proba(test_embeddings)

# Analyze errors
print("\n" + "="*80)
print("ERROR ANALYSIS")
print("="*80)

errors = []
for i in range(len(test_subset)):
    if predictions[i] != test_subset["label"][i]:
        confidence = probabilities[i][predictions[i]]
        errors.append({
            'index': i,
            'text': test_subset["text"][i],
            'true_label': test_subset["label"][i],
            'predicted_label': predictions[i],
            'confidence': confidence,
            'length': len(test_subset["text"][i].split())
        })

total_errors = len(errors)
total_samples = len(test_subset)
accuracy = (total_samples - total_errors) / total_samples

print(f"\nOverall Performance:")
print(f"  Correct: {total_samples - total_errors}/{total_samples} ({accuracy:.1%})")
print(f"  Errors:  {total_errors}/{total_samples} ({total_errors/total_samples:.1%})")

# Categorize errors
false_positives = [e for e in errors if e['predicted_label'] == 1]
false_negatives = [e for e in errors if e['predicted_label'] == 0]

print(f"\nError Types:")
print(f"  False Positives: {len(false_positives)} (predicted positive, actually negative)")
print(f"  False Negatives: {len(false_negatives)} (predicted negative, actually positive)")

# Show high-confidence errors (most surprising)
high_conf_errors = [e for e in errors if e['confidence'] > 0.7]

print(f"\n" + "-"*80)
print(f"HIGH-CONFIDENCE ERRORS (confidence > 0.7)")
print(f"These are the most surprising mistakes:")
print("-"*80)

for i, error in enumerate(high_conf_errors[:5]):
    true_sent = "Positive" if error['true_label'] == 1 else "Negative"
    pred_sent = "Positive" if error['predicted_label'] == 1 else "Negative"

    print(f"\n{i+1}. '{error['text']}'")
    print(f"   True: {true_sent} | Predicted: {pred_sent} | Confidence: {error['confidence']:.3f}")
    print(f"   Length: {error['length']} words")

# Analyze by text length
print(f"\n" + "-"*80)
print("ERROR ANALYSIS BY TEXT LENGTH")
print("-"*80)

error_lengths = [e['length'] for e in errors]
correct_lengths = [len(test_subset["text"][i].split())
                   for i in range(len(test_subset))
                   if predictions[i] == test_subset["label"][i]]

avg_error_length = np.mean(error_lengths) if error_lengths else 0
avg_correct_length = np.mean(correct_lengths) if correct_lengths else 0

print(f"\nAverage length of ERROR reviews: {avg_error_length:.1f} words")
print(f"Average length of CORRECT reviews: {avg_correct_length:.1f} words")

if avg_error_length < avg_correct_length:
    print(f" Observation: Errors tend to be SHORTER")
elif avg_error_length > avg_correct_length:
    print(f" Observation: Errors tend to be LONGER")
else:
    print(f" Observation: No clear length pattern")

# Test edge cases
print("\n" + "="*80)
print("TESTING EDGE CASES")
print("="*80)

edge_cases = [
    ("Sarcastic", "Oh great, another masterpiece. NOT!", 0),
    ("Mixed", "The acting was great but the plot was terrible", 0),
    ("Backhanded", "Not as bad as I expected", 1),
    ("Double negative", "Not unwatchable", 1),
    ("Very short", "Boring", 0),
    ("Ambiguous", "It was a movie", 0),
]

# TODO: After analyzing above errors, add your own test cases:
# edge_cases.extend([
#     ("Your category", "Your test review here", expected_label_0_or_1),
#     ("Another category", "Another test review", expected_label),
# ])

print("\nTesting challenging cases that often fail:")
print("-"*80)

edge_embeddings = model.encode([text for _, text, _ in edge_cases])
edge_predictions = clf.predict(edge_embeddings)
edge_probs = clf.predict_proba(edge_embeddings)

correct_count = 0
for i, (category, text, true_label) in enumerate(edge_cases):
    pred = edge_predictions[i]
    conf = edge_probs[i][pred]
    correct = pred == true_label
    if correct:
        correct_count += 1

    true_sent = "Positive" if true_label == 1 else "Negative"
    pred_sent = "Positive" if pred == 1 else "Negative"

    print(f"\n{category}: '{text}'")
    print(f"  True: {true_sent} | Predicted: {pred_sent} | Confidence: {conf:.3f}")
    print(f"  Result: {' CORRECT' if correct else ' WRONG'}")

edge_accuracy = correct_count / len(edge_cases)
print(f"\n" + "-"*80)
print(f"Edge Case Accuracy: {correct_count}/{len(edge_cases)} ({edge_accuracy:.1%})")
print(f"Regular Test Accuracy: {accuracy:.1%}")
print(f"Difference: {accuracy - edge_accuracy:+.1%}")

# Summary and insights
print("\n" + "="*80)
print("KEY INSIGHTS FROM ERROR ANALYSIS")
print("="*80)

print("\n1. Error Distribution:")
print(f"   - False Positives (predicted too optimistic): {len(false_positives)}")
print(f"   - False Negatives (predicted too pessimistic): {len(false_negatives)}")
if len(false_positives) > len(false_negatives):
    print(f"    Classifier has POSITIVE BIAS")
elif len(false_negatives) > len(false_positives):
    print(f"    Classifier has NEGATIVE BIAS")

print("\n2. Challenging Cases:")
failing_categories = [cat for cat, text, true in edge_cases
                     if clf.predict(model.encode([text]))[0] != true]
if failing_categories:
    print(f"   The classifier struggles with: {', '.join(failing_categories)}")

print("\n3. Confidence Analysis:")
if high_conf_errors:
    print(f"   Found {len(high_conf_errors)} high-confidence errors")
    print(f"    The model is 'confidently wrong' on some cases")

print("\n" + "="*80)
print("TODO: Based on your error analysis, propose improvements:")
print("="*80)
print("# Write your observations here:")
print("# 1. What patterns did you notice in the errors?")
print("# 2. Which edge cases failed most?")
print("# 3. How would you improve the classifier?")
print("#    - Better training data?")
print("#    - Different features?")
print("#    - Ensemble approach?")
print("#    - Confidence thresholds?")

Training classifier on 1000 movie reviews...

ERROR ANALYSIS

Overall Performance:
  Correct: 174/200 (87.0%)
  Errors:  26/200 (13.0%)

Error Types:
  False Positives: 11 (predicted positive, actually negative)
  False Negatives: 15 (predicted negative, actually positive)

--------------------------------------------------------------------------------
HIGH-CONFIDENCE ERRORS (confidence > 0.7)
These are the most surprising mistakes:
--------------------------------------------------------------------------------

1. 'an uneasy mix of run-of-the-mill raunchy humor and seemingly sincere personal reflection .'
   True: Negative | Predicted: Positive | Confidence: 0.701
   Length: 13 words

2. 'the stunt work is top-notch ; the dialogue and drama often food-spittingly funny .'
   True: Negative | Predicted: Positive | Confidence: 0.867
   Length: 14 words

3. 'goldmember is funny enough to justify the embarrassment of bringing a barf bag to the moviehouse .'
   True: Positive | Predicted:

### Questions

1. What do high-confidence errors have in common?

2. Do errors tend to be shorter or longer than correct predictions?

3. Which edge cases failed most - sarcasm, mixed sentiment, or double negatives?
